In [28]:
import pickle
import os

from pymongo import MongoClient

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

# Download Links

In [29]:
with open('ted_video_links_2730.pkl', 'rb') as picklefile:
    links_video_ted = pickle.load(picklefile)

In [79]:
#links_video_ted

# Set up DB

In [31]:
client = MongoClient()
db = client['ted']
db.create_collection("ted_stats")
ted_stats = db.get_collection('ted_stats')

# Get Statistics with API

In [3]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "../client_secret_comedy-project-276920.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    
    for url in links_video_ted:
        try:
            request = youtube.videos().list(part="snippet,contentDetails,statistics",id=url)
            response = request.execute()
            ted_stats.insert_one(response)
        except:
            continue

if __name__ == "__main__":
    main()

In [89]:
ted_stats.count()

/Users/elena/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


2732

In [49]:
list(ted_stats.find_one())

['_id', 'kind', 'etag', 'items', 'pageInfo']

In [70]:
#cursor = db.ted_stats.find({}, {'_id':0, 'items.id': 1})
#list(cursor)

In [ ]:
item["id"]["lowered_name"]

In [65]:
scraped = []
for item in db.ted_stats.find({}, {'_id':0, 'items.id': 1}):
    #print(item['items'])
    for id_ in item['items']:
        scraped.append(id_['id'])
        
scraped

In [68]:
to_scrape = list(set(links_video_ted) - set(scraped))

In [78]:
len(links_video_ted) - (len(scraped) + len(to_scrape))

-2

In [85]:
len(to_scrape)

1188

In [86]:
ted_stats.find_one()

{'_id': ObjectId('5ebde861c8aec9d0ff43480e'),
 'kind': 'youtube#videoListResponse',
 'etag': 'ozuCto5nyuXvUBW0gfSWj2V5JPI',
 'items': [{'kind': 'youtube#video',
   'etag': '8hFc92dcK-oh0zO7sK0eQzeynqs',
   'id': 'Bb7kz1THJPU',
   'snippet': {'publishedAt': '2020-04-20T13:28:47Z',
    'channelId': 'UCAuUUnT6oDeKwE6v1NGQxug',
    'title': 'Go ahead, dream about the future | Charlie Jane Anders',
    'description': 'Visit http://TED.com to get our entire library of TED Talks, transcripts, translations, personalized talk recommendations and more.\n\n"You don\'t predict the future -- you imagine the future," says sci-fi writer Charlie Jane Anders. In a talk that\'s part dream, part research-based extrapolation, she takes us on a wild, speculative tour of the delights and challenges the future may hold -- and shows how dreaming up weird, futuristic possibilities empowers us to construct a better tomorrow.\n\nThe TED Talks channel features the best talks and performances from the TED Conferen

## Second iteration because of google limitations

In [1]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def main():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "../client_secret_comedy-project-276920.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    
    for url in to_scrape:
        try:
            request = youtube.videos().list(part="snippet,contentDetails,statistics",id=url)
            response = request.execute()
            ted_stats.insert_one(response)
        except:
            continue

if __name__ == "__main__":
    main()